In [1]:
# ! pip install openpyxl xlsxwriter requests pandas tqdm

In [2]:
%load_ext autoreload
%autoreload 1
%aimport main_functions

import requests, pandas as pd, os
from time import sleep
from datetime import datetime
from tqdm import tqdm
from main_functions import subset_table, list_to_rows, save_as_excel, clean_bank_account, org_name_clean 
from data.data_for_downloader.renamers import renamer_1, renamer_2, renamer_3, renamer_4, renamer_5, renamer_6, renamer_7, renamer_8, renamer_9, renamer_10, renamer_11, renamer_12, renamer_13, renamer_14, renamer_15, renamer_16, renamer_17, renamer_18, renamer_19, renamer_20

# pandas show all columns 
pd.set_option('display.max_columns', None)

# pandas wider columns
pd.set_option('display.max_colwidth', None)

In [3]:
# !!! set to True if you want to update all data from the very beginning
full_update = True

https://politdata.nazk.gov.ua/api/swagger/#/reports/get_getreport__id_  
https://politdata.nazk.gov.ua/#/  
https://docs.google.com/document/d/1X68OvBYCQEiGG_nbjiD2fE0PZ1JIvbSsfXjrHUjOZ0c/edit?usp=sharing  

## Download party info

In [4]:
# таблиця зі всіма партіями (api/getpartylistmain)

party_list = requests.get('https://politdata.nazk.gov.ua/api/getpartylistmain').json()
party_list = pd.DataFrame(party_list['data'])
party_list = party_list.drop(['locationFact','politPartyUnitId','locations'], axis=1)


# таблиця зі всіма регіональними представництвами (api/getpartylistregion)

party_region_list = requests.get('https://politdata.nazk.gov.ua/api/getpartylistregion').json()
party_region_list = pd.DataFrame(party_region_list['data'])
party_region_list = party_region_list.drop(['locationFact','locations'], axis=1) 

# додати інформацію про центральні партії до регіональних філій
party_region_list = party_region_list.merge(party_list[['name','unitId','code']]
                                            .rename({'name':'party_main_name', 'code':'party_main_EDRPOU','unitId':'party_main_unitId'}, 
                                                    axis=1), 
                                            right_on='party_main_unitId', 
                                            left_on='politPartyUnitId', 
                                            how='left')

# перейменувати
party_region_list = party_region_list.rename({'name':'local_org_name', 'code':'local_org_EDRPOU'},axis=1)

In [5]:
# Якщо серед донорів зустрічаються партійні осередки чи центральний офіс політичної партії 'donor_type' замінити на “партійний осередок” (визначила по 'donor_edrpou')
def check_edrpou_for_party(table, var_edrpou_to_search, var_to_replace):
    table.loc[(table[var_edrpou_to_search].isin(party_list.code.tolist()))|
              (table[var_edrpou_to_search].isin(party_region_list.local_org_EDRPOU.tolist())), 
              var_to_replace] = 'Партійний осередок'
    return table

## Download all reports

In [6]:
# завантажити список всіх звітів, які є в системі

report_list = requests.get('https://politdata.nazk.gov.ua/api/getreportslist').json()
report_list = [X['id'] for X in report_list]
len(report_list)

813

In [7]:
# завантажити список всіх звітів, які вже були завантажені

with open('data/data_for_downloader/downloaded_report_ids.txt', 'r') as f:
    downloaded_report_ids = f.read().splitlines()

len(downloaded_report_ids)

812

In [8]:
# у залежності від типу оновлення (повний чи частковий) визначити список звітів, які потрібно завантажити

if full_update:
    reports_to_download = report_list
else:
    reports_to_download = [x for x in report_list if x not in downloaded_report_ids]

len(reports_to_download)

813

In [9]:
r_df = pd.DataFrame()
for i in tqdm(reports_to_download):
    one_report = requests.get(f'https://politdata.nazk.gov.ua/api/getreport/{i}').json()
    
    one_report_df = pd.DataFrame(one_report)
    one_report_df['report_id'] = i
    r_df = pd.concat([r_df, one_report_df], axis=0, ignore_index=True)

    sleep(0.2)

100%|██████████| 813/813 [08:28<00:00,  1.60it/s]  


In [10]:
# Файл зі всіма завантаженими звітами

if full_update:

    # створити новий файл зі всіма завантаженими звітами
    with open('data/data_for_downloader/downloaded_report_ids.txt', 'w') as f:
        for report_id in r_df['report_id'].unique():
            f.write(str(report_id) + '\n')

else:
    
    # додати підвантажені звіти до txt файлу
    with open('data/data_for_downloader/downloaded_report_ids.txt', 'a') as f:
        for report_id in r_df['report_id'].unique():
            f.write(str(report_id) + '\n')

In [11]:
# додати інфу про центральний офіс (який був визначений в кожному звіті)
party_main_finder = r_df.loc[r_df.officeType=="Центральний офіс",['report_id','partyCode','partyName']].rename({'partyCode':'party_main_EDRPOU','partyName':'party_main_name'}, axis=1)

# обʼєднати r_df та party_main_finder
r_df = r_df.merge(party_main_finder, on='report_id', how='left')

In [12]:
# Чистка назв партій

r_df['party_main_name'] = r_df['party_main_name'].str.upper()

to_delete = ['^ПОЛІТИЧНА ПАРТІЯ ', '^ВСЕУКРАЇНСЬКЕ ОБ\'ЄДНАННЯ ', 'ВСЕУКРАЇНСЬКЕ ПОЛІТИЧНЕ ОБ\'ЄДНАННЯ',
             'ПОЛІТИЧЯНА ПАРТІЯ','СОЦІАЛЬНО-ЕКОЛОГІЧНА ПАРТІЯ', 'ВСЕУКРАЇНСЬКЕ ОБ\'ЄДНАННЯ', 'СОЦІАЛЬНО-ПОЛІТИЧНИЙ СОЮЗ ',
             '«', '»', '\"']
for words in to_delete:
    r_df['party_main_name'] = r_df['party_main_name'].str.replace(words, '', regex=True)

# прибрати зайві пробіли
r_df['party_main_name'] = r_df['party_main_name'].str.replace('\s+', ' ', regex=True).str.strip()


# ручні правки
party_renamer = {
    'РІШУЧИХ ДІЙ': 'ПАРТІЯ РІШУЧИХ ДІЙ',
    'КОНКРЕТНИХ СПРАВ':'ПАРТІЯ КОНКРЕТНИХ СПРАВ',
    'БЛОК СВІТЛИЧНОЇ РАЗОМ!':'БЛОК СВІТЛИЧНОЇ «РАЗОМ!»',
    'ПАРТІЯ ВОЛОДИМИРА БУРЯКА ЄДНАННЯ':'ПАРТІЯ ВОЛОДИМИРА БУРЯКА «ЄДНАННЯ»',
    'МИР':'ВСЕУКРАЇНСЬКЕ ОБ\'ЄДНАННЯ «МИР»',
    'ПАРТІЯ ІГОРЯ КОЛИХАЄВА НАМ ТУТ ЖИТИ!':'ПАРТІЯ ІГОРЯ КОЛИХАЄВА «НАМ ТУТ ЖИТИ!»',
    'КОМАНДА МАКСИМА ЄФІМОВА НАШ КРАМАТОРСЬК':'КОМАНДА МАКСИМА ЄФІМОВА «НАШ КРАМАТОРСЬК»',
    'ГРОМАДСЬКО-ПОЛІТИЧНИЙ РУХ ВАЛЕНТИНА НАЛИВАЙЧЕНКА СПРАВЕДЛИВІСТЬ':'ГРОМАДСЬКО-ПОЛІТИЧНИЙ РУХ ВАЛЕНТИНА НАЛИВАЙЧЕНКА «СПРАВЕДЛИВІСТЬ»',
    'МАЛОГО І СЕРЕДНЬОГО БІЗНЕСУ УКРАЇНИ':'ПАРТІЯ МАЛОГО І СЕРЕДНЬОГО БІЗНЕСУ УКРАЇНИ',
    'БЛОК ВІЛКУЛА УКРАЇНСЬКА ПЕРСПЕКТИВА':'БЛОК ВІЛКУЛА «УКРАЇНСЬКА ПЕРСПЕКТИВА»',
    'НАЦІОНАЛЬНО-ДЕМОКРАТИЧНЕ ОБ\'ЄДНАННЯ УКРАЇНА':'НАЦІОНАЛЬНО-ДЕМОКРАТИЧНЕ ОБ\'ЄДНАННЯ «УКРАЇНА»',
    'ГРОМАДЯНСЬКИЙ РУХ СВІДОМІ':'ГРОМАДЯНСЬКИЙ РУХ «СВІДОМІ»',
    'ВО ПЛАТФОРМА ГРОМАД':'ПЛАТФОРМА ГРОМАД'
}

r_df['party_main_name'] = r_df['party_main_name'].replace(party_renamer)

## Tables

### Таблиця 0.1: Перевірка на наявність дублів

In [13]:
# create report about duplicates
to_subset = ['period','year','partyName','partyCode','report_id']
duplicates = r_df[r_df.duplicated(subset=to_subset,keep=False)]

if len(duplicates) > 0:
    save_as_excel(duplicates, '0_report_duplcates', full_update)

In [14]:
len(duplicates)

642

### Таблиця 1: загальна інформація про партію/осередок

In [15]:
filename = '1_legal_entity_report_info'

In [16]:
cols_to_select = [x for x in renamer_1.keys() if x in r_df.columns]
table = r_df[cols_to_select].rename(renamer_1, axis=1)

# add None if empty column
for v in renamer_1.values():
    if v not in table.columns:
        table[v] = None

table = table[list(renamer_1.values())]

save_as_excel(table, filename, full_update)

### Таблиця 0.2: Інформація про те, за який період кожна партія подала звіти

In [17]:
table = pd.read_excel('data/excel_tables/1_legal_entity_report_info.xlsx')

table_subset = table[['legal_entity_name','legal_entity_edrpou','officeType','party_main_name','party_main_EDRPOU','report_id','report_period','report_year']]
table_subset['reported_period'] = table_subset['report_year'].astype(str) + ', ' + table_subset['report_period']
table_subset = table_subset.drop(['report_period','report_year'], axis=1).drop_duplicates()

# якщо для 1 періоду є декілька report_id - об'єднати через ;  
table_subset = table_subset.groupby(['legal_entity_name','legal_entity_edrpou','officeType','party_main_name','party_main_EDRPOU','reported_period'], as_index=False).agg({'report_id': lambda x: '; '.join(x)})

table_subset = table_subset.pivot(index=['legal_entity_name','legal_entity_edrpou','officeType','party_main_name','party_main_EDRPOU'], 
                                  values='report_id',columns='reported_period',).reset_index().sort_values('party_main_name').fillna('')

/tmp/ipykernel_104793/371056082.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  table_subset['reported_period'] = table_subset['report_year'].astype(str) + ', ' + table_subset['report_period']


In [18]:
table_subset.to_excel('data/excel_tables/0_reports_per_period_per_party.xlsx', index=False, engine='xlsxwriter')

### Таблиця 2: інформація про осередки політичної партії  
Назва файлу: 2.2_other_party_orgs_info

In [19]:
main_var = 'tablets1'

cols_to_select = ['partyLocationRegion','period', 'year', 'party_main_name','party_main_EDRPOU','report_id', 'officeType']

filename = '2.1_local_orgs_info'

In [20]:
table = subset_table(r_df, main_var, cols_to_select)

if len(table) > 0:

    t = list_to_rows(table, main_var, cols_to_select, renamer_2)
    t = t[list(renamer_2.values())+['party_main_name','party_main_EDRPOU','report_id','officeType']]

    # Для центральних офісів політичних партій legal_entity_region –> “Україна”
    t.loc[t.officeType == 'Центральний офіс', 'legal_entity_region'] = 'Україна'
    # і потім видалити 'officeType'
    t = t.drop('officeType', axis=1)

    save_as_excel(t, filename, full_update)

### Таблиця 3: інформація про інші підприємства, установи та організації створені партією політичної партії

In [21]:
cols_to_select = ['period', 'year', 'party_main_name','party_main_EDRPOU','report_id']

main_var = 'tablets2'

filename = '2.2_other_party_orgs_info'

In [22]:
table = subset_table(r_df, main_var, cols_to_select)

if len(table) > 0:
    t = list_to_rows(table, main_var, cols_to_select, renamer_3)
    t = t[list(renamer_3.values())+['party_main_name','party_main_EDRPOU','report_id']]
    
    # delete empty values
    t = t[(t.other_party_org_name != '0')|(t.other_party_org_EDRPOU != '00000000')]

    save_as_excel(t, filename, full_update)

### Таблиця 4: зведена інформація про нерухоме майно політичної партії (propertyObjects)

In [23]:
cols_to_select = ['officeType','types','period', 'year','partyName','partyCode','partyLocationRegion', 
                  'party_main_name','party_main_EDRPOU','report_id']

main_var = 'propertyObjects'

filename = '3.1_property_objects'

In [24]:
table = subset_table(r_df, main_var, cols_to_select)

if len(table) > 0:
    t = list_to_rows(table, main_var, cols_to_select, renamer_4)
    t = t[list(renamer_4.values())+['party_main_name','party_main_EDRPOU','report_id']]

    # delete empty
    t = t[t.object_type.notna()]

    # Для центральних офісів політичних партій legal_entity_region –> “Україна”
    t.loc[t.officeType == 'Центральний офіс', 'legal_entity_region'] = 'Україна'

    # clean t['object_owner_name']

    if len(t.object_owner_name.unique()) > 1:

        t['object_owner_name'] = t['object_owner_name'].str.upper()
        t['object_owner_name'] = t['object_owner_name'].str.replace('\s+',' ', regex=True).str.strip()

        # уніфікувати ФОП, ТОВ і ПАТ
        org_name_clean(t, 'object_owner_name')

        # уніфікувати апострофи в іменах
        t['object_owner_name'] = t['object_owner_name'].replace(r"(?<=\w)[’\"\`](?=\w)", "\'", regex=True)

        # replace ocassional english letters
        t.loc[t['object_owner_name'].str.contains('[А-Я]', na=False),'object_owner_name'] = t.loc[t['object_owner_name'].str.contains('[А-Я]', na=False),'object_owner_name'].str.replace('C','С').str.replace('I','І')

        save_as_excel(t, filename, full_update)

### Таблиця 5: зведена інформація про рухоме майно політичної партії (propertyMovables)

In [25]:
cols_to_select = ['officeType','types','period', 'year','partyName','partyCode', 'partyLocationRegion', 
                  'party_main_name','party_main_EDRPOU','report_id']

main_var = 'propertyMovables'

filename = '3.2_movable_property'

In [26]:
table = subset_table(r_df, main_var, cols_to_select)

if len(table) > 0:
    t = list_to_rows(table, main_var, cols_to_select, renamer_5)
    
    # Для центральних офісів політичних партій legal_entity_region –> “Україна”
    t.loc[t.officeType == 'Центральний офіс', 'legal_entity_region'] = 'Україна'

    # sort columns
    sorter = [x for x in list(renamer_5.values())+['party_main_name','party_main_EDRPOU','report_id']  if x in t.columns]
    t = t[sorter]

    # clean t['object_owner_name']
    var_to_clean = 'object_owner'

    t[var_to_clean] = t[var_to_clean].str.upper()
    t[var_to_clean] = t[var_to_clean].str.replace('\s+',' ', regex=True).str.strip()

    # уніфікувати ФОП, ТОВ і ПАТ
    org_name_clean(t, var_to_clean)

    # уніфікувати апострофи в іменах
    t[var_to_clean] = t[var_to_clean].replace(r"(?<=\w)[’\"\`](?=\w)", "\'", regex=True)

    # replace ocassional english letters
    t.loc[t[var_to_clean].str.contains('[А-Я]', na=False),var_to_clean] = t.loc[t[var_to_clean].str.contains('[А-Я]', na=False),var_to_clean].str.replace('C','С').str.replace('I','І')

    save_as_excel(t, filename, full_update)


### Таблиця 6: зведена інформація про автотранспорт політичної партії (propertyTransport)

In [27]:
cols_to_select = ['officeType','types','period', 'year','partyName','partyCode', 'partyLocationRegion', 
                  'party_main_name','party_main_EDRPOU','report_id']

main_var = 'propertyTransport'

filename = '3.3_vehicles'

In [28]:
table = subset_table(r_df, main_var, cols_to_select)

if len(table) > 0:

    t = list_to_rows(table, main_var, cols_to_select, renamer_6)

    # sort columns
    sorter = [x for x in list(renamer_6.values())+['party_main_name','party_main_EDRPOU','report_id']  if x in t.columns]
    t = t[sorter]

    # Для центральних офісів політичних партій legal_entity_region –> “Україна”
    t.loc[t.officeType == 'Центральний офіс', 'legal_entity_region'] = 'Україна'

    # delete empty
    t = t[(t.object_type.notna())&(t.object_brand.notna())]

    # Якщо object_owner_edrpou це код партії чи осередку то у object_owner_type ставити “Партійний осередок”
    t = check_edrpou_for_party(t, 'object_owner_edrpou', 'object_owner_type')

    # clean t['object_owner']
    var_to_clean = 'object_owner'
    t[var_to_clean] = t[var_to_clean].str.upper()
    t[var_to_clean] = t[var_to_clean].str.replace('\s+',' ', regex=True).str.strip()

    # уніфікувати ФОП, ТОВ і ПАТ
    org_name_clean(t, var_to_clean)

    # уніфікувати апострофи в іменах
    t[var_to_clean] = t[var_to_clean].replace(r"(?<=\w)[’\"\`](?=\w)", "\'", regex=True)

    # replace ocassional english letters
    t.loc[t[var_to_clean].str.contains('[А-Я]', na=False),var_to_clean] = t.loc[t[var_to_clean].str.contains('[А-Я]', na=False),var_to_clean].str.replace('C','С').str.replace('I','І')

    save_as_excel(t, filename, full_update)


### Таблиця 7: зведена інформація про цінні папери політичної партії (propertyPapers)

In [29]:
cols_to_select = ['officeType','types','period', 'year','partyName','partyCode', 'partyLocationRegion',
                  'party_main_name','party_main_EDRPOU','report_id']

main_var = 'propertyPapers'

filename = '3.4_securities'

In [30]:
table = subset_table(r_df, main_var, cols_to_select)

if len(table) > 0:
    t = list_to_rows(table, main_var, cols_to_select, renamer_7)

    if len(t) > 0:
        # sort columns
        sorter = [x for x in list(renamer_7.values())+['party_main_name','party_main_EDRPOU','report_id']  if x in t.columns]
        t = t[sorter]

        # Для центральних офісів політичних партій legal_entity_region –> “Україна”
        t.loc[t.officeType == 'Центральний офіс', 'legal_entity_region'] = 'Україна'

        # save
        save_as_excel(t, filename, full_update)


    elif len(t) == 0:
        # create columns if t is empty
        t = pd.DataFrame(columns=list(renamer_7.values())+['party_main_name','party_main_EDRPOU','report_id'])
        save_as_excel(t, filename, full_update)

else:
    # create columns if table is empty
    t = pd.DataFrame(columns=list(renamer_7.values())+['party_main_name','party_main_EDRPOU','report_id'])
    save_as_excel(t, filename, full_update)
    
    
    

### Таблиця 8: зведена інформація про інше майно та нематеріальні активи політичної партії (propertyNoMoney)

In [31]:
cols_to_select = ['officeType','types','period', 'year','partyName','partyCode', 'partyLocationRegion', 
                  'party_main_name','party_main_EDRPOU','report_id']

main_var = 'propertyNoMoney'

filename = '3.5_intangible_assets'

In [32]:
table = subset_table(r_df, main_var, cols_to_select)
if len(table) > 0:

    t = list_to_rows(table, main_var, cols_to_select, renamer_8)

    # Для центральних офісів політичних партій legal_entity_region –> “Україна”
    t.loc[t.officeType == 'Центральний офіс', 'legal_entity_region'] = 'Україна'

    save_as_excel(t, filename, full_update)

### Таблиця 9: зведена інформація про банківські рахунки політичної партії (propertyMoney)

In [33]:
cols_to_select = ['officeType','types','period', 'year','partyName','partyCode', 'partyLocationRegion', 
                  'party_main_name','party_main_EDRPOU','report_id']

main_var = 'propertyMoney'

filename = '4_bank_accounts'

In [34]:
table = subset_table(r_df, main_var, cols_to_select)

if len(table) > 0:

    t = list_to_rows(table, main_var, cols_to_select, renamer_9)

    t['account_number'] = clean_bank_account(t['account_number'])

    # check if any are longer than 29
    t['account_number'].apply(lambda x: len(x)).value_counts()

    # t[t.account_number.apply(lambda x: len(x)>29)]

    # Для центральних офісів політичних партій legal_entity_region –> “Україна”
    t.loc[t.officeType == 'Центральний офіс', 'legal_entity_region'] = 'Україна'

    save_as_excel(t, filename, full_update)


### Таблиця 10: зведена інформація про приватні грошові внески на рахунки політичної партії (contributionConMoney)  

In [35]:
cols_to_select = ['officeType','types','period', 'year','partyName','partyCode', 'party_main_name','party_main_EDRPOU',
                  'partyLocationRegion','report_id']

main_var = 'contributionConMoney'

filename = '5_private_contributions'

In [36]:
table = subset_table(r_df, main_var, cols_to_select)
if len(table) > 0:
    t = list_to_rows(table, main_var, cols_to_select, renamer_10)

    # delete empty
    t = t[(t.donor_name.notna())&(t.bank_edrpou.notna())]

    # clean bank_account
    t['bank_account'] = clean_bank_account(t['bank_account'])

    # Якщо серед донорів зустрічаються партійні осередки чи центральний офіс політичної партії 'donor_type' замінити на “партійний осередок” (визначила по 'donor_edrpou')
    t = check_edrpou_for_party(t, 'donor_edrpou', 'donor_type')

    # Для центральних офісів політичних партій legal_entity_region –> “Україна”
    t.loc[t.officeType == 'Центральний офіс', 'legal_entity_region'] = 'Україна'

    # sort columns
    t = t[list(renamer_10.values())+['party_main_name','party_main_EDRPOU','report_id']]

    # clean t['donor_name']
    var_to_clean = 'donor_name'

    t[var_to_clean] = t[var_to_clean].str.upper()
    t[var_to_clean] = t[var_to_clean].str.replace('\s+',' ', regex=True).str.strip()

    # уніфікувати ФОП, ТОВ і ПАТ
    org_name_clean(t, var_to_clean)

    # уніфікувати апострофи в іменах
    t[var_to_clean] = t[var_to_clean].replace(r"(?<=\w)[’\"\`](?=\w)", "\'", regex=True)

    # replace ocassional english letters
    t.loc[t[var_to_clean].str.contains('[А-Я]', na=False),var_to_clean] = t.loc[t[var_to_clean].str.contains('[А-Я]', na=False),var_to_clean].str.replace('C','С').str.replace('I','І')

    save_as_excel(t, filename, full_update)

### Таблиця 11: зведена інформація про негрошові внески на користь політичної партії (contributionOtherCon)

In [37]:

cols_to_select = ['officeType','types','period', 'year','partyName','partyCode', 'party_main_name','party_main_EDRPOU','partyLocationRegion','report_id']

main_var = 'contributionOtherCon'

filename = '6_in_kind_donations'

In [38]:
table = subset_table(r_df, main_var, cols_to_select)
if len(table) > 0:

    t = list_to_rows(table, main_var, cols_to_select, renamer_11)

    # delete empty
    t = t[(t.donor_name.notna())&(t.donor_type.notna())]

    # Якщо серед донорів зустрічаються партійні осередки чи центральний офіс політичної партії 'donor_type' замінила на “партійний осередок” (визначила по 'donor_edrpou')

    t.loc[(t.donor_edrpou.isin(party_list.code.tolist()))|
        (t.donor_edrpou.isin(party_region_list.local_org_EDRPOU.tolist())), 
        'donor_type'] = 'Партійний осередок'

    # donor_edrpou delete *
    t.loc[t.donor_edrpou.str.contains('\*',na=False),'donor_edrpou'] = None
    t.loc[t.donor_birth_date.str.contains('\*',na=False),'donor_birth_date'] = None
    t.loc[t.object_registration_number.str.contains('\*',na=False),'object_registration_number'] = None

    # Для центральних офісів політичних партій legal_entity_region –> “Україна”

    t.loc[t.officeType == 'Центральний офіс', 'legal_entity_region'] = 'Україна'

    # clean t['donor_name']
    var_to_clean = 'donor_name'

    t[var_to_clean] = t[var_to_clean].str.upper()
    t[var_to_clean] = t[var_to_clean].str.replace('\s+',' ', regex=True).str.strip()

    # уніфікувати ФОП, ТОВ і ПАТ
    org_name_clean(t, var_to_clean)

    # уніфікувати апострофи в іменах
    t[var_to_clean] = t[var_to_clean].replace(r"(?<=\w)[’\"\`](?=\w)", "\'", regex=True)

    # replace ocassional english letters
    t.loc[t[var_to_clean].str.contains('[А-Я]', na=False),var_to_clean] = t.loc[t[var_to_clean].str.contains('[А-Я]', na=False),var_to_clean].str.replace('C','С').str.replace('I','І')

    save_as_excel(t, filename, full_update)

### Таблиця 12: зведена інформація про кошти державного фінансування виплачені політичним партіям (contributionCosts)
Завдання: об'єднати всі дані щодо коштів державного фінансування отриманих
партіями. Також, потрібно почистити costs3/bank_account (пробіли, зайві символи).  
  
Назва файлу: 7_state_funding_transactions

In [39]:
cols_to_select = ['officeType','types','period', 'year','partyName','partyCode', 'party_main_name','party_main_EDRPOU','report_id']

main_var = 'contributionCosts'

filename = '7_state_funding_transactions'

In [40]:
table = subset_table(r_df, main_var, cols_to_select)
if len(table) > 0:
    t = list_to_rows(table, main_var, cols_to_select, renamer_12)

    # convert refund_sum to numeric
    t['transaction_sum'] = t['transaction_sum'].astype(int)

    t['bank_account'] = clean_bank_account(t['bank_account'])

    # sort columns
    t = t[list(renamer_12.values()) + ['party_main_name','party_main_EDRPOU','report_id']]

    save_as_excel(t, filename, full_update)

### Таблиця 13: зведена інформація про інші грошові надходження на рахунки політичних партій (contributionOtherCosts)

In [41]:
cols_to_select = ['officeType','types','period', 'year','partyName','partyCode', 'partyLocationRegion',
                  'party_main_name','party_main_EDRPOU','report_id']

main_var = 'contributionOtherCosts'

filename = '8_other_income'

In [42]:
table = subset_table(r_df, main_var, cols_to_select)
if len(table) > 0:

    t = list_to_rows(table, main_var, cols_to_select, renamer_13)
    t['bank_account'] = clean_bank_account(t['bank_account'])

    # Для центральних офісів політичних партій legal_entity_region –> “Україна”
    t.loc[t.officeType == 'Центральний офіс', 'legal_entity_region'] = 'Україна'

    # clean t['sender_name']
    var_to_clean = 'sender_name'

    t[var_to_clean] = t[var_to_clean].str.upper()
    t[var_to_clean] = t[var_to_clean].str.replace('\s+',' ', regex=True).str.strip()

    # уніфікувати ФОП, ТОВ і ПАТ
    org_name_clean(t, var_to_clean)

    # уніфікувати апострофи в іменах
    t[var_to_clean] = t[var_to_clean].replace(r"(?<=\w)[’\"\`](?=\w)", "\'", regex=True)

    # replace ocassional english letters
    t.loc[t[var_to_clean].str.contains('[А-Я]', na=False),var_to_clean] = t.loc[t[var_to_clean].str.contains('[А-Я]', na=False),var_to_clean].str.replace('C','С').str.replace('I','І')

    save_as_excel(t, filename, full_update)

### Таблиця 14: зведена інформація про витрати державного фінансування політичних партій (paymentGov)

In [43]:
cols_to_select = ['officeType','types','period', 'year','partyName','partyCode', 'party_main_name','party_main_EDRPOU',
                  'partyLocationRegion','report_id']

main_var = 'paymentGov'

filename = '9.1_expenditures_public_funding'

In [44]:
table = subset_table(r_df, main_var, cols_to_select)
if len(table) > 0:

    t = list_to_rows(table, main_var, cols_to_select, renamer_14)

    t['account_number'] = clean_bank_account(t['account_number'])

    # Для центральних офісів політичних партій legal_entity_region –> “Україна”
    t.loc[t.officeType == 'Центральний офіс', 'legal_entity_region'] = 'Україна'

    # Якщо отримувач коштів є партійною структурою (recipient_EDRPOU), у recipient_type замінити позначку “Юридична особа/Фізична особа” на Партійний осередок.
    t = check_edrpou_for_party(t, 'recipient_EDRPOU', 'recipient_type')

    # clean t['recipient_name']
    var_to_clean = 'recipient_name'
    t[var_to_clean] = t[var_to_clean].str.upper()
    t[var_to_clean] = t[var_to_clean].str.replace('\s+',' ', regex=True).str.strip()

    # уніфікувати ФОП, ТОВ і ПАТ
    org_name_clean(t, var_to_clean)

    # уніфікувати апострофи в іменах
    t[var_to_clean] = t[var_to_clean].replace(r"(?<=\w)[’\"\`](?=\w)", "\'", regex=True)

    # replace ocassional english letters
    t.loc[t[var_to_clean].str.contains('[А-Я]', na=False),var_to_clean] = t.loc[t[var_to_clean].str.contains('[А-Я]', na=False),var_to_clean].str.replace('C','С').str.replace('I','І')

    save_as_excel(t, filename, full_update)

### Таблиця 15: зведена інформація про витрати з рахунків з приватним фінансуванням (paymentOther)

In [45]:
cols_to_select = ['officeType','types','period', 'year','partyName','partyCode', 'party_main_name','party_main_EDRPOU','partyLocationRegion','report_id']

main_var = 'paymentOther'

filename = '9.2_expenditures_private_funds'

In [46]:
table = subset_table(r_df, main_var, cols_to_select)
if len(table) > 0:

    t = list_to_rows(table, main_var, cols_to_select, renamer_15)

    t['account_number'] = clean_bank_account(t['account_number'])

    # Для центральних офісів політичних партій legal_entity_region –> “Україна”
    t.loc[t.officeType == 'Центральний офіс', 'legal_entity_region'] = 'Україна'

    # Якщо отримувач коштів є партійною структурою (recipient_EDRPOU), у recipient_type замінити позначку “Юридична особа/Фізична особа” на Партійний осередок.
    t = check_edrpou_for_party(t, 'recipient_EDRPOU', 'recipient_type')

    # delete empty
    t = t[(t.bank_EDRPOU.notna())&(t.account_number.notna())]

    # clean t['recipient_name']
    var_to_clean = 'recipient_name'
    t[var_to_clean] = t[var_to_clean].str.upper()
    t[var_to_clean] = t[var_to_clean].str.replace('\s+',' ', regex=True).str.strip()

    # уніфікувати ФОП, ТОВ і ПАТ
    org_name_clean(t, var_to_clean)

    # уніфікувати апострофи в іменах
    t[var_to_clean] = t[var_to_clean].replace(r"(?<=\w)[’\"\`](?=\w)", "\'", regex=True)

    # replace ocassional english letters
    t.loc[t[var_to_clean].str.contains('[А-Я]', na=False),var_to_clean] = t.loc[t[var_to_clean].str.contains('[А-Я]', na=False),var_to_clean].str.replace('C','С').str.replace('I','І')

    save_as_excel(t, filename, full_update)

### Таблиця 16: зведена інформація про отримання помилкових надходжень (paymentCostsPaymentReceive)

In [47]:
cols_to_select = ['officeType','types','period', 'year','partyName','partyCode', 'partyLocationRegion',
                  'party_main_name','party_main_EDRPOU','report_id']

main_var = 'paymentCostsPaymentReceive'

filename = '9.3_false_donations_info'

In [48]:
table = subset_table(r_df, main_var, cols_to_select)
if len(table) > 0:

    t = list_to_rows(table, main_var, cols_to_select, renamer_16)

    t['account_number'] = clean_bank_account(t['account_number'])

    # Для центральних офісів політичних партій legal_entity_region –> “Україна”
    t.loc[t.officeType == 'Центральний офіс', 'legal_entity_region'] = 'Україна'

    # clean t['donor_name']
    var_to_clean = 'donor_name'
    t[var_to_clean] = t[var_to_clean].str.upper()
    t[var_to_clean] = t[var_to_clean].str.replace('\s+',' ', regex=True).str.strip()

    # уніфікувати ФОП, ТОВ і ПАТ
    org_name_clean(t, var_to_clean)

    # уніфікувати апострофи в іменах
    t[var_to_clean] = t[var_to_clean].replace(r"(?<=\w)[’\"\`](?=\w)", "\'", regex=True)

    # replace ocassional english letters
    t.loc[t[var_to_clean].str.contains('[А-Я]', na=False),var_to_clean] = t.loc[t[var_to_clean].str.contains('[А-Я]', na=False),var_to_clean].str.replace('C','С').str.replace('I','І')

    save_as_excel(t, filename, full_update)

### НЕ ІСНУЄ Таблиця 17: інформація про повернення внесків та помилкових надходжень (paymentCostsPaymentReturn)
Завдання: обєднати всю інформацію про повернення помилкових внесків.  
  
Назва файлу: 9.4_false_donations_returning

In [49]:
cols_to_select = ['officeType','types','period', 'year','partyName','partyCode', 'party_main_name','party_main_EDRPOU','report_id']

main_var = 'paymentCostsPaymentReturn'

filename = '9.4_false_donations_returning'

In [50]:
main_var in r_df.columns

False

### Таблиця 18: інформація про отримання помилкових інших (негрошових) внесків (paymentOtherCostsPaymentReceive)

In [51]:
cols_to_select = ['officeType','types','period', 'year','partyName','partyCode', 'partyLocationRegion', 
                  'party_main_name','party_main_EDRPOU','report_id']

main_var = 'paymentOtherCostsPaymentReceive'

filename = '9.5_false_in_kind_donations_info'

In [52]:
table = subset_table(r_df, main_var, cols_to_select)
if len(table) > 0:
    t = list_to_rows(table, main_var, cols_to_select, renamer_18)

    if len(t) == 0:
        # create columns if t is empty
        t = pd.DataFrame(columns=list(renamer_18.values())+['party_main_name','party_main_EDRPOU','report_id'])
        save_as_excel(t, filename, full_update)

    else:
        # sort columns
        sorter = [x for x in list(renamer_18.values())+['party_main_name','party_main_EDRPOU','report_id']  if x in t.columns]
        t = t[sorter]

        # Для центральних офісів політичних партій legal_entity_region –> “Україна”
        t.loc[t.officeType == 'Центральний офіс', 'legal_entity_region'] = 'Україна'

        # clean t['recipient_name']

        var_to_clean = 'onor_name'

        t[var_to_clean] = t[var_to_clean].str.upper()
        t[var_to_clean] = t[var_to_clean].str.replace('\s+',' ', regex=True).str.strip()

        # уніфікувати ФОП, ТОВ і ПАТ
        org_name_clean(t, var_to_clean)

        # уніфікувати апострофи в іменах
        t[var_to_clean] = t[var_to_clean].replace(r"(?<=\w)[’\"\`](?=\w)", "\'", regex=True)

        # replace ocassional english letters
        t.loc[t[var_to_clean].str.contains('[А-Я]', na=False),var_to_clean] = t.loc[t[var_to_clean].str.contains('[А-Я]', na=False),var_to_clean].str.replace('C','С').str.replace('I','І')

        # save
        save_as_excel(t, filename, full_update)

else:
    # create t
    t = pd.DataFrame(columns=list(renamer_18.values())+['party_main_name','party_main_EDRPOU','report_id'])
    save_as_excel(t, filename, full_update)
        

### НЕ ІСНУЄ Таблиця 19: інформація про отримання помилкових інших (негрошових) внесків (paymentOtherCostsPaymentReturn)

In [53]:
cols_to_select = ['officeType','types','period', 'year','partyName','partyCode', 'party_main_name','party_main_EDRPOU','report_id']

main_var = 'paymentOtherCostsPaymentReturn'

filename = '9.5_false_in_kind_donations_returning'

In [54]:
main_var in r_df.columns

False

### Таблиця 20: інформація про фінансові зобов'язання (obligate)

In [55]:
cols_to_select = ['officeType','types','period', 'year','partyName','partyCode', 'partyLocationRegion',
                  'party_main_name','party_main_EDRPOU','report_id']

main_var = 'obligate'

filename = '10_liabilities'

In [56]:
table = subset_table(r_df, main_var, cols_to_select)
if len(table) > 0:
    t = list_to_rows(table, main_var, cols_to_select, renamer_20)

    # Для центральних офісів політичних партій legal_entity_region –> “Україна”
    t.loc[t.officeType == 'Центральний офіс', 'legal_entity_region'] = 'Україна'

    # delete empty
    t = t[t.name.notnull()]

    # clean t['name']
    var_to_clean = 'name'
    t[var_to_clean] = t[var_to_clean].str.upper()
    t[var_to_clean] = t[var_to_clean].str.replace('\s+',' ', regex=True).str.strip()

    # уніфікувати ФОП, ТОВ і ПАТ
    org_name_clean(t, var_to_clean)

    # уніфікувати апострофи в іменах
    t[var_to_clean] = t[var_to_clean].replace(r"(?<=\w)[’\"\`](?=\w)", "\'", regex=True)

    # replace ocassional english letters
    t.loc[t[var_to_clean].str.contains('[А-Я]', na=False),var_to_clean] = t.loc[t[var_to_clean].str.contains('[А-Я]', na=False),var_to_clean].str.replace('C','С').str.replace('I','І')

    save_as_excel(t, filename, full_update)


## Перевірити в яких файлах є які партії

In [71]:
df = pd.read_excel('data/excel_tables/1_legal_entity_report_info.xlsx')[['legal_entity_name','legal_entity_edrpou','officeType','party_main_name','party_main_EDRPOU']]
df = df.drop_duplicates()

df

,legal_entity_name,legal_entity_edrpou,officeType,party_main_name,party_main_EDRPOU
0,ПОЛІТИЧНА ПАРТІЯ «ІДЕЯ НАЦІЇ»,43724237,Центральний офіс,ІДЕЯ НАЦІЇ,43724237
1,ПОЛІТИЧНА ПАРТІЯ «ПРОГРЕСИВНА СОЦІАЛІСТИЧНА ПАРТІЯ УКРАЇНИ»,13497,Центральний офіс,ПРОГРЕСИВНА СОЦІАЛІСТИЧНА ПАРТІЯ УКРАЇНИ,13497
2,ПОЛІТИЧНА ПАРТІЯ «ПАТРІОТИЧНА ПАРТІЯ УКРАЇНИ»,21676167,Центральний офіс,ПАТРІОТИЧНА ПАРТІЯ УКРАЇНИ,21676167
3,ПОЛІТИЧНА ПАРТІЯ «ЗА ОДЕЩИНУ»,40212900,Центральний офіс,ЗА ОДЕЩИНУ,40212900
4,ПОЛІТИЧНА ПАРТІЯ «ВАРТА (ВСЕУКРАЇНСЬКИЙ АЛЬЯНС РЕГІОНАЛЬНИХ І ТЕРИТОРІАЛЬНИХ АКТИВІСТІВ)»,43648015,Центральний офіс,ВАРТА (ВСЕУКРАЇНСЬКИЙ АЛЬЯНС РЕГІОНАЛЬНИХ І ТЕРИТОРІАЛЬНИХ АКТИВІСТІВ),43648015
...,...,...,...,...,...
13422,ЧОПСЬКОЇ МІСЬКОЇ ОРГАНІЗАЦІЇ ПОЛІТИЧНОЇ ПАРТІЇ «НАРОДНО- ТРУДОВИЙ СОЮЗ УКРАЇНИ»,38273440,Регіональний офіс,НАРОДНО-ТРУДОВИЙ СОЮЗ УКРАЇНИ,34349353
13670,Мелітопольська міська організація Партії Зелених України,25891371,Регіональний офіс,ПАРТІЯ ЗЕЛЕНИХ УКРАЇНИ,47728
14586,ПОЛІТИЧНА ПАРТІЯ «НАРОДНО-ДЕМОКРАТИЧНА ПАРТІЯ (НДП)»,13267,Центральний офіс,НАРОДНО-ДЕМОКРАТИЧНА ПАРТІЯ (НДП),13267
14591,"Політична партія Дніпропетровська обласна огранізація політичної партії ""Разом Сила""",44254310,Регіональний офіс,РАЗОМ СИЛА,40469495


In [73]:
excel_files = os.listdir('data/excel_tables')
excel_files = [x for x in excel_files if x.endswith('.xlsx') and not x.startswith('0_')]
len(excel_files)

18

## Last updates

In [57]:
# Додати в readme дату і час останнього оновлення

now = datetime.now().strftime("%Y-%m-%d %H:%M")

with open('README.md', 'r') as file:
    data = file.readlines()
    data.append(f"\n\nОстаннє оновлення: {now}\n")
with open('README.md', 'w') as file:
    file.writelines(data)

In [58]:
print('Done')

Done


## Check quality of tables

In [64]:
filename = '0_report_duplcates'
df = pd.read_excel(f'data/excel_tables/{filename}.xlsx')
df.head(1)

,date,year,types,period,website,obligate,tablets1,tablets2,isWebsite,partyCode,partyName,documentId,officeType,paymentGov,headLastName,paymentOther,reportNumber,headFirstName,propertyMoney,quantityFirst,quantityThird,headMiddleName,propertyPapers,quantitySecond,signerFullName,paymentOtherSum,propertyNoMoney,propertyObjects,headLocationCity,propertyMovables,contributionCosts,partyLocationCity,propertyTransport,headLocationRegion,partyLocationIndex,headLocationCountry,partyIsLocationSame,partyLocationRegion,contributionConMoney,contributionOtherCon,partyLocationCountry,partyLocationBuilding,partyLocationDistrict,contributionOtherCosts,partyLocationCountryFact,paymentCostsPaymentReceive,paymentOtherCostsPaymentReceive,id,report_id,headLocationApt,partyLocationApt,partyLocationAptFact,partyLocationIndexFact,partyLocationKorpusFact,partyLocationRegionFact,partyLocationBuildingFact,partyLocationDistrictFact,partyLocationStreet,headIdNumber,headIdIssurer,isAddedToCentralOffice,partyLocationCityFact,partyLocationStreetFact,headProxyDoc,headLocationKorpus,partyLocationKorpus,headLocationAptFact,headLocationIndexFact,headLocationRegionFact,headLocationStreetFact,headLocationCountryFact,headLocationBuildingFact,headLocationDistrictFact,signatures,headLocationCityFact,headLocationKorpusFact,headLocationForeignLocationFact,party_main_EDRPOU,party_main_name
0,2021-09-28T14:07:54.559Z,2021,Основний,2 квартал,NaN,[],[],[],NaN,35846609,Кременецька районна партійна організація Аграрної партії України,a9d0c230-2061-11ec-9c82-eb6359fa003a,Регіональний офіс,[],Лаць,[],31310,Андрій,[],1.0,1.0,Іванович,[],0.0,NaN,0.0,[],[],с. Серединки,[],[],с. Білокриниця,[],Тернопільська обл.,47013,Україна,Так,Тернопільська обл.,[],[],Україна,34,Кременецький р-н,[],Україна,[],[],7a122d00-2065-11ec-945d-214b0bac8091,b78a8d80-00eb-11ef-95c4-e7bfad33aba2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,вул. Вишнева,NaN,NaN,Додано до звіту центрального офісу,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,34763401,АГРАРНА ПАРТІЯ УКРАЇНИ


In [65]:
df.party_main_name.value_counts().head(10)

party_main_name
ПАРТІЯ ЗЕЛЕНИХ УКРАЇНИ           512
АГРАРНА ПАРТІЯ УКРАЇНИ            60
САМОВРЯДНА УКРАЇНСЬКА ДЕРЖАВА     40
ПАРТІЯ СИЛЬНОЇ ВЛАДИ              30
Name: count, dtype: int64